<a href="https://colab.research.google.com/github/sakuna47/AI_Maze/blob/Code/AI_MAZE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random  # For random selection of start, goal, and barriers
import math  # For calculating diagonal costs (sqrt(2))
import heapq  # For priority queue in UCS and A* search
import statistics  # For calculating mean and variance in analysis
from collections import deque  # For BFS in solvability check

In [2]:
# Convert a node number (0-35) to (x, y) coordinates in a 6x6 grid
# x = node % 6 (column), y = node // 6 (row)
# Example: Node 15 -> (x=2, y=3)
def get_coordinates(node):
    return (node % 6, node // 6)


In [3]:
# Get valid neighbors of a node with their edge costs
# Neighbors are in 8 directions: horizontal, vertical, diagonal
# Barriers block movement; neighbors are sorted by node number for deterministic exploration
def get_neighbors(node, barriers):
    x, y = get_coordinates(node)  # Get current node's coordinates
    # Define 8 possible directions: up, down, left, right, and diagonals
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (-1, 1), (1, -1), (1, 1)]
    neighbors = []
    for dx, dy in directions:
        new_x, new_y = x + dx, y + dy  # Calculate neighbor's coordinates
        # Check if the neighbor is within the 6x6 grid
        if 0 <= new_x < 6 and 0 <= new_y < 6:
            neighbor = new_y * 6 + new_x  # Convert coordinates back to node number
            if neighbor not in barriers:  # Exclude barriers
                # Cost is 1 for horizontal/vertical moves, sqrt(2) for diagonal moves
                cost = 1 if dx == 0 or dy == 0 else math.sqrt(2)
                neighbors.append((neighbor, cost))
    # Sort neighbors by node number to ensure consistent exploration order
    # Example: For node 8, neighbors should be processed as 2, 7, 9, 14
    neighbors.sort(key=lambda x: x[0])
    return neighbors

In [4]:

# Use BFS to check if a path exists between start and goal nodes
# Ensures the maze is solvable before proceeding with search algorithms
def is_solvable(start, goal, barriers):
    visited = set()  # Track visited nodes
    queue = deque([start])  # Queue for BFS
    visited.add(start)
    while queue:
        node = queue.popleft()  # Process the next node
        if node == goal:  # Path found
            return True
        # Explore neighbors of the current node
        for neighbor, _ in get_neighbors(node, barriers):
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append(neighbor)
    return False  # No path found

In [5]:
# Task 1: Set up a 6x6 maze with random start, goal, and barriers
# Start node: 0-11, Goal node: 24-35, Barriers: 4 random nodes
# Ensures the maze is solvable using BFS
def setup_maze():
    while True:
        start = random.randint(0, 11)  # Random start node from 0-11
        goal = random.randint(24, 35)  # Random goal node from 24-35
        # Get remaining nodes (excluding start and goal) for barrier selection
        remaining = [i for i in range(36) if i != start and i != goal]
        barriers = random.sample(remaining, 4)  # Select 4 barriers
        # Check if a path exists from start to goal
        if is_solvable(start, goal, barriers):
            return start, goal, barriers

In [6]:

# Task 3: Calculate Chebyshev Distance heuristic for A* search
# Formula: max(|Nx - Gx|, |Ny - Gy|)
# Used to estimate the remaining distance to the goal
def chebyshev_distance(node, goal):
    x1, y1 = get_coordinates(node)  # Current node's coordinates
    x2, y2 = get_coordinates(goal)  # Goal node's coordinates
    return max(abs(x1 - x2), abs(y1 - y2))  # Chebyshev Distance